In [1]:
class DICApriori:
    def __init__(self, transactions, min_support):
        self.transactions = transactions
        self.min_support = min_support
        self.item_counts = {}
        self.freq_itemsets = []

    def generate_candidates(self, prev_itemsets, k):
        candidates = []
        for i in range(len(prev_itemsets)):
            for j in range(i + 1, len(prev_itemsets)):
                itemset1 = prev_itemsets[i]
                itemset2 = prev_itemsets[j]
                if itemset1[:-1] == itemset2[:-1]:
                    new_candidate = itemset1 + (itemset2[-1],)
                    candidates.append(new_candidate)
        return candidates

    def count_itemsets(self, itemsets):
        counts = {}
        for transaction in self.transactions:
            for itemset in itemsets:
                if set(itemset).issubset(set(transaction)):
                    counts[itemset] = counts.get(itemset, 0) + 1
        return counts

    def fit(self):
        # Count individual items
        for transaction in self.transactions:
            for item in transaction:
                self.item_counts[item] = self.item_counts.get(item, 0) + 1

        # Find frequent 1-itemsets
        freq_1_itemsets = [(item,) for item, count in self.item_counts.items() if count >= self.min_support]
        self.freq_itemsets.append(freq_1_itemsets)

        k = 2
        while len(self.freq_itemsets[k - 2]) > 0:
            # Generate candidate itemsets
            candidates = self.generate_candidates(self.freq_itemsets[k - 2], k)

            # Count support for candidates
            candidate_counts = self.count_itemsets(candidates)

            # Keep only frequent itemsets
            freq_itemsets = [itemset for itemset, count in candidate_counts.items() if count >= self.min_support]

            # Add frequent itemsets to list
            self.freq_itemsets.append(freq_itemsets)

            k += 1

    def get_frequent_itemsets(self):
        return self.freq_itemsets

transactions = [
    [1, 2, 5],
    [2, 4],
    [2, 3],
    [1, 2, 4],
    [1, 3],
    [2, 3],
    [1, 3],
    [1, 2, 3, 5],
    [1, 2, 3]
]
min_support = 2
apriori = DICApriori(transactions, min_support)
apriori.fit()
frequent_itemsets = apriori.get_frequent_itemsets()
print("Frequent Itemsets:")
for k, itemsets in enumerate(frequent_itemsets, start=1):
    print(f"k={k}: {itemsets}")

Frequent Itemsets:
k=1: [(1,), (2,), (5,), (4,), (3,)]
k=2: [(1, 2), (1, 5), (2, 5), (2, 4), (2, 3), (1, 3)]
k=3: [(1, 2, 5), (1, 2, 3)]
k=4: []


In [2]:
from collections import defaultdict

class HashApriori:
    def __init__(self, transactions, min_support):
        self.transactions = transactions
        self.min_support = min_support
        self.item_counts = defaultdict(int)
        self.freq_itemsets = []

    def generate_candidates(self, prev_itemsets, k):
        candidates = []
        for i in range(len(prev_itemsets)):
            for j in range(i + 1, len(prev_itemsets)):
                itemset1 = prev_itemsets[i]
                itemset2 = prev_itemsets[j]
                if itemset1[:-1] == itemset2[:-1]:
                    new_candidate = itemset1 + (itemset2[-1],)
                    candidates.append(new_candidate)
        return candidates

    def prune_candidates(self, candidates, k):
        pruned_candidates = []
        for candidate in candidates:
            subsets = [candidate[:i] + candidate[i + 1:] for i in range(k)]
            if all(tuple(subset) in self.freq_itemsets[k - 2] for subset in subsets):
                pruned_candidates.append(candidate)
        return pruned_candidates

    def scan_transactions(self, candidates):
        counts = defaultdict(int)
        for transaction in self.transactions:
            for candidate in candidates:
                if all(item in transaction for item in candidate):
                    counts[candidate] += 1
        return counts

    def fit(self):
        # Count individual items
        for transaction in self.transactions:
            for item in transaction:
                self.item_counts[item] += 1

        # Find frequent 1-itemsets
        freq_1_itemsets = [(itemset,) for itemset, count in self.item_counts.items() if count >= self.min_support]
        self.freq_itemsets.append(freq_1_itemsets)

        k = 2
        while len(self.freq_itemsets[k - 2]) > 0:
            # Generate candidate itemsets
            candidates = self.generate_candidates(self.freq_itemsets[k - 2], k)

            # Prune candidates
            pruned_candidates = self.prune_candidates(candidates, k)

            # Count support for pruned candidates
            candidate_counts = self.scan_transactions(pruned_candidates)

            # Keep only frequent itemsets
            freq_itemsets = [itemset for itemset, count in candidate_counts.items() if count >= self.min_support]

            # Add frequent itemsets to list
            self.freq_itemsets.append(freq_itemsets)
            k += 1

    def get_frequent_itemsets(self):
        return self.freq_itemsets

transactions = [
    (1, 2, 5),
    (2, 4),
    (2, 3),
    (1, 2, 4),
    (1, 3),
    (2, 3),
    (1, 3),
    (1, 2, 3, 5),
    (1, 2, 3)
]
min_support = 2

apriori = HashApriori(transactions, min_support)
apriori.fit()
frequent_itemsets = apriori.get_frequent_itemsets()
print("Frequent Itemsets:")
for k, itemsets in enumerate(frequent_itemsets, start=1):
    print(f"k={k}: {itemsets}")


Frequent Itemsets:
k=1: [(1,), (2,), (5,), (4,), (3,)]
k=2: [(1, 2), (1, 5), (2, 5), (2, 4), (2, 3), (1, 3)]
k=3: [(1, 2, 5), (1, 2, 3)]
k=4: []
